<a href="https://colab.research.google.com/github/russellemergentai/FineTune_Run_MistralInstruct/blob/main/JackBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jack Bot, T4 is fine.

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets scipy
!pip install -q trl
!pip install -q langchain
!pip install -U langchain-community
!pip install -q sentence_transformers
!pip install -q faiss-cpu

from google.colab import drive
drive.mount("/content/drive")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline
import re

# load model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V5"
llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left', add_eos_token=True)

# Define common pipeline parameters
common_params = {
    "torch_dtype": torch.bfloat16,
    "trust_remote_code": True,
    "device_map": {"": 0},
    "num_return_sequences": 1,
    "repetition_penalty": 1.1
}

# Initialize pipelines with specific parameters
pipelineBase = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    max_length=500,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    **common_params,
    max_new_tokens=128
)

pipelineSUMMARY = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    max_length=2000,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    **common_params
)

# Create HuggingFacePipeline objects
llmPipeline = HuggingFacePipeline(pipeline=pipelineBase, model_kwargs={"temperature": 0.5})
llmPipelineSummary = HuggingFacePipeline(pipeline=pipelineSUMMARY, model_kwargs={"temperature": 0.1})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

<ipython-input-2-354c341675b2>:52: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llmPipeline = HuggingFacePipeline(pipeline=pipelineBase, model_kwargs={"temperature": 0.5})


In [3]:
# create retriever
from langchain.document_loaders import TextLoader
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def create_retriever(path):
    if "https" in path:
      loader = WebBaseLoader(path)
    else:
      loader = TextLoader(path, encoding='UTF-8')

    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000, chunk_overlap=1000, separators=[" ", ",", "\n"]
    )

    texts = text_splitter.split_documents(documents)#
    model_path="intfloat/e5-large-unsupervised"

    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': False}
    )

    db = FAISS.from_documents(texts, embeddings)
    retriever = db.as_retriever()
    return retriever


def extract_text(text):
    # Split the text by '<s>'
    parts = text.split('<s>')

    # Check if there are multiple parts
    if len(parts) > 1:
        # Return the text after the final '<s>' tag
        return remove_strings(parts[-1].strip())
    else:
        # If '<s>' is not found, return an empty string or the original text
        return ""

def remove_strings(text):
    strings_to_remove = ["[INST]", "[/INST]", "</s>", "<s>"]
    for string in strings_to_remove:
        text = text.replace(string, "")
    return text.rstrip()

def remove_text(text, n: int=5):
    # Split the text into parts
    parts = text.split("</s>")

    # Extract the comments section
    if len(parts) > 1:
        comments_section = parts[1].strip()
    else:
        comments_section = ""

    # Split the comments into individual lines
    comments = comments_section.splitlines()

    # Get the first four lines
    lines = comments[:n]

    # Join and print the first four lines
    result = "\n".join(lines).strip()

    return result

from langchain.chains import RetrievalQA, SequentialChain
from google.colab import output
from langchain import PromptTemplate

# retrieve from data.txt or http page and run query
def retrieval_query():

  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>
   <s>
  """

# use default path if none is passed, else use a web link i.e. # path="https://en.wikipedia.org/wiki/Footfall"
  path = input("enter http path or leave empty for default data file load: ")
  if (path==''): path="/content/drive/MyDrive/Target/data.txt"

  retriever = create_retriever(path)
  output.clear()

  query = input("enter your query on the uploaded data: ")

  qa = RetrievalQA.from_chain_type(llm=llmPipelineSummary, retriever=retriever, return_source_documents=False)
  result = qa.run({"query": query})
  result = remove_strings(result)
  print("RESULT_BASE>" + result)


# summarize from data.txt
def summary_query():

  # Chain of summary
  summarize_template = """<s>[INST] The following is an article:
  {query}
    You will generate increasingly concise, entity-dense summaries of the above Article.

    Repeat the following 2 steps 3 times.

    Set NW=100

    Step 1. Identify 1-3 informative Entities ("; " delimited) from the Article which are missing from the previously generated summary.
    Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

    A Missing Entity is:
    - Relevant: to the main story.
    - Specific: descriptive yet concise (5 words or fewer).
    - Novel: not in the previous summary.
    - Faithful: present in the Article.
    - Anywhere: located anywhere in the Article.

    Guidelines:
    - The first summary should be long (4-5 sentences, ~NW words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~NW words.
    - Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
    - Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
    - The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
    - Missing entities can appear anywhere in the new summary.
    - Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

    Remember, use the exact same number of words for each summary.
  [/INST]
  </s>
  <s>
  """

  path = "/content/drive/MyDrive/Target/data.txt"
  SUMMARIZE = PromptTemplate.from_template(summarize_template)
  llm_chain = SUMMARIZE | llmPipelineSummary

  loader = TextLoader(path, encoding = 'UTF-8')
  text = loader.load()

  result = llm_chain.invoke({"query": text})
  result = remove_strings(result)
  print("RESULT_BASE>" + result)


# query the base model with a commentary from the fine tuned model
def standard_query():
  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>
   <s>
  """

  query = input("enter the llm query: ")

  PROMPT = PromptTemplate.from_template(prompt_template)
  llm_chain = PROMPT | llmPipeline
  result = llm_chain.invoke({"query": query})
  result = extract_text(result)
  print("BASE>" + result)



In [6]:

# main loop
def main():
    functions = {'1': standard_query, '2': retrieval_query, '3': summary_query}

    while True:
        user_input = input("Enter 1 llm query, 2 data or web retrieval query, 3 data summary query or 'x' to exit: ").lower()
        output.clear()

        if user_input in functions:
            functions[user_input]()
        elif user_input == 'x':
            print("Exiting.")
            break
        else:
            print("Invalid input, please try again.")

if __name__ == "__main__":
    main()

Exiting.
